### Tratando outliers, mascarando dados sensíveis e normalizando os dados

In [1]:
import os
import pandas as pd
import plotly.express as px
import numpy as np
import warnings
import json
from collections import defaultdict
from sklearn.preprocessing import MinMaxScaler

# Configurações do pandas e warnings
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")

# ==============================================================================
# SEÇÃO 1: DEFINIÇÃO DE CAMINHOS (PORTÁTIL: COLAB/LOCAL)
# ==============================================================================
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    drive.mount('/content/drive')
    caminho_base = "/content/drive/MyDrive/Estudos/Mestrado/Projeto de Mestrado/"
else:
    caminho_base = "G:/Meu Drive/Estudos/Mestrado/Projeto de Mestrado/"

# Define caminhos para os dados brutos e para os dados tratados
caminho_dados_originais = os.path.join(caminho_base, "bases_originais")
caminho_dados_tratados = os.path.join(caminho_base, "bases_tratadas")

# Cria a pasta de saída se ela não existir
os.makedirs(caminho_dados_tratados, exist_ok=True)

# ==============================================================================
# SEÇÃO 2: FUNÇÕES AUXILIARES PARA LIMPEZA DE DADOS
# ==============================================================================

def corrigir_sequencias_de_zeros(df_grupo, coluna, min_sequencia=2):
    # Encontra sequências de zeros (>= min_sequencia) e as preenche usando interpolação linear.
    df = df_grupo.copy()
    is_zero = df[coluna] == 0
    blocos = (is_zero != is_zero.shift()).cumsum()
    tamanho_blocos = df.groupby(blocos)[coluna].transform('size')
    condicao = (is_zero) & (tamanho_blocos >= min_sequencia)
    df.loc[condicao, coluna] = np.nan
    df[coluna] = df[coluna].interpolate(method='linear')
    df[coluna] = df[coluna].ffill().bfill()
    return df

def filtrar_e_imputar_spikes_robusto(df_grupo, coluna, threshold_pico=1.5):
    # Corrige picos extremos e zeros isolados que sobraram.
    df = df_grupo.copy()
    prev_val = df[coluna].shift(1)
    next_val = df[coluna].shift(-1)
    media_vizinhos = (prev_val + next_val) / 2
    condicao_pico = (df[coluna] > prev_val * threshold_pico) & (df[coluna] > next_val * threshold_pico) & (prev_val > 0) & (next_val > 0)
    condicao_queda = (df[coluna] == 0) & (prev_val.notna()) & (prev_val != 0) & (next_val.notna()) & (next_val != 0)
    is_outlier = condicao_pico | condicao_queda
    df[coluna] = np.where(is_outlier, media_vizinhos, df[coluna])
    df[coluna].fillna(df_grupo[coluna], inplace=True)
    return df

# ==============================================================================
# SEÇÃO 3: PIPELINE PRINCIPAL DE PROCESSAMENTO DE DADOS
# ==============================================================================

print("\n--- INICIANDO PIPELINE DE PROCESSAMENTO ---")

# --- 3.1 Carregamento dos Dados Brutos ---
print("ETAPA 1/7: Carregando dados brutos...")
try:
    df = pd.read_csv(os.path.join(caminho_dados_originais, "Medicoes_2018-2024.csv"), sep=',', encoding='latin-1', skiprows=1)
except FileNotFoundError:
    print(f"ERRO: Arquivo 'Medicoes_2018-2024.csv' não encontrado no caminho especificado.")
    exit() # Encerra o script se o arquivo principal não for encontrado

# --- 3.2 Geração e Carregamento do Dicionário de Mapeamento ---
print("ETAPA 2/7: Gerando ou carregando o mapeamento de transformadores...")
caminho_mapeamento = os.path.join(caminho_dados_originais, "mapeamento_trafos.json")

if not os.path.exists(caminho_mapeamento):
    print("  -> Arquivo de mapeamento não encontrado. Gerando um novo...")
    trafos_originais = df['Equipamento Medição'].unique()
    grupos = defaultdict(list)
    for trafo in sorted(trafos_originais):
        sigla = trafo[:3]
        grupos[sigla].append(trafo)
    dicionario_mascarado = {}
    contador_t = 1
    alfabeto = 'abcdefghijklmnopqrstuvwxyz'
    for sigla in sorted(grupos.keys()):
        trafos_no_grupo = grupos[sigla]
        if len(trafos_no_grupo) == 1:
            dicionario_mascarado[trafos_no_grupo[0]] = f"T{contador_t}"
        else:
            for i, nome_original in enumerate(trafos_no_grupo):
                dicionario_mascarado[nome_original] = f"T{contador_t}{alfabeto[i]}"
        contador_t += 1
    with open(caminho_mapeamento, 'w', encoding='utf-8') as f:
        json.dump(dicionario_mascarado, f, ensure_ascii=False, indent=4)
else:
    print("  -> Arquivo de mapeamento encontrado. Carregando...")
    with open(caminho_mapeamento, 'r', encoding='utf-8') as f:
        dicionario_mascarado = json.load(f)

# --- 3.3 Pré-processamento Inicial ---
print("ETAPA 3/7: Realizando pré-processamento inicial...")
df.rename(columns={'Potência Ativa': 'P', 'Potência Reativa': 'Q', 'Data/Hora Medição': 'datahora', 'Equipamento Medição': 'id'}, inplace=True)
df['id'] = df['id'].map(dicionario_mascarado)
df['P'] = pd.to_numeric(df['P'].str.replace(',', '.'), errors='coerce')
df['Q'] = pd.to_numeric(df['Q'].str.replace(',', '.'), errors='coerce')
df['datahora'] = pd.to_datetime(df['datahora'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
df.dropna(subset=['id', 'datahora', 'P', 'Q'], inplace=True)
df['S'] = np.sqrt(df['P'].abs()**2 + df['Q'].abs()**2)
df = df[['id', 'datahora', 'S']]

# --- 3.4 Limpeza de Outliers ---
print("ETAPA 4/7: Aplicando filtros de limpeza de outliers...")
# ETAPA A: Corrigir as sequências de zeros
df_etapa_1 = df.groupby('id').apply(lambda group: corrigir_sequencias_de_zeros(group, 'S')).reset_index(drop=True)
# ETAPA B: Corrigir os spikes isolados
df_limpo = df_etapa_1.groupby('id').apply(lambda group: filtrar_e_imputar_spikes_robusto(group, 'S')).reset_index(drop=True)

# --- 3.5 Correção Condicional de Escala ---
print("ETAPA 5/7: Aplicando correção de escala (>50)...")
df_limpo.loc[df_limpo['S'] > 50, 'S'] /= 10

# --- 3.6 Normalização Min-Max por Transformador ---
print("ETAPA 6/7: Normalizando a demanda (Min-Max)...")
df_limpo['S_normalizado'] = df_limpo.groupby('id')['S'].transform(
    lambda x: MinMaxScaler().fit_transform(x.values.reshape(-1, 1)).flatten()
)

# --- 3.7 Preparação do DataFrame Final e Salvamento ---
print("ETAPA 7/7: Gerando e salvando o dataset final...")
df_hourly = df_limpo[['id', 'datahora', 'S_normalizado']].copy()
df_hourly.rename(columns={'S_normalizado': 'S'}, inplace=True) # Renomeia para 'S' para simplicidade

caminho_saida_csv = os.path.join(caminho_dados_tratados, 'transformers_dataset.csv')
df_hourly.to_csv(caminho_saida_csv, index=False, sep=';')

print(f"\n--- PIPELINE CONCLUÍDO ---")


--- INICIANDO PIPELINE DE PROCESSAMENTO ---
ETAPA 1/7: Carregando dados brutos...
ETAPA 2/7: Gerando ou carregando o mapeamento de transformadores...
  -> Arquivo de mapeamento encontrado. Carregando...
ETAPA 3/7: Realizando pré-processamento inicial...
ETAPA 4/7: Aplicando filtros de limpeza de outliers...
ETAPA 5/7: Aplicando correção de escala (>50)...
ETAPA 6/7: Normalizando a demanda (Min-Max)...
ETAPA 7/7: Gerando e salvando o dataset final...

--- PIPELINE CONCLUÍDO ---


### Ajustando para pegar a potência pico por dia

In [2]:
df_hourly['datahora'] = pd.to_datetime(df_hourly['datahora'], format='%Y-%m-%d %H:%M:%S')

df_daily = df_hourly.groupby('id', group_keys=False).apply(lambda x: x.set_index('datahora').resample('D').max()).reset_index()
df_daily.to_csv(r"G:\Meu Drive\Estudos\Mestrado\Github\masters\bases_tratadas\daily_peak_transformers_dataset.csv", index=False, sep=';')
